In [1]:
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
from gensim.summarization import summarize
from rank_bm25 import BM25Okapi
from nltk import sent_tokenize, word_tokenize
from konlpy.tag import Hannanum
from konlpy.tag import Kkma
hannanum = Hannanum()
import pandas as pd
import time ,re , timeit ,os
from datetime import datetime
from konlpy.tag import Okt

In [2]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [2]:
start_now = int(time.time())
td = datetime.today().strftime("%Y%m%d")
tdd = datetime.today().strftime("%m%d")
now = datetime.now()
tdnow = now.strftime('%Y%m%d%H%M%S')

In [3]:
## 코사인 유사도의 단어로 BM25 방식을 이용하여 유사도 텍스트 문장을 추출한다. 
### from gensimBM25Source 
import math
from six import iteritems
from six.moves import xrange
# BM25 parameters.
PARAM_K1 = 1.5
PARAM_B = 0.75
EPSILON = 0.25

In [4]:
class BM25(object):
    def __init__(self, corpus):
        self.corpus_size = len(corpus)
        self.avgdl = sum(map(lambda x: float(len(x)), corpus)) / self.corpus_size
        self.corpus = corpus
        self.f = []
        self.df = {}
        self.idf = {}
        self.initialize()

    def initialize(self):
        for document in self.corpus:
            frequencies = {}
            for word in document:
                if word not in frequencies:
                    frequencies[word] = 0
                frequencies[word] += 1
            self.f.append(frequencies)

            for word, freq in iteritems(frequencies):
                if word not in self.df:
                    self.df[word] = 0
                self.df[word] += 1

        for word, freq in iteritems(self.df):
            self.idf[word] = math.log(self.corpus_size - freq + 0.5) - math.log(freq + 0.5)

    def get_score(self, document, index, average_idf):
        score = 0
        for word in document:
            if word not in self.f[index]:
                continue
            idf = self.idf[word] if self.idf[word] >= 0 else EPSILON * average_idf
            score += (idf * self.f[index][word] * (PARAM_K1 + 1)
                      / (self.f[index][word] + PARAM_K1 * (1 - PARAM_B + PARAM_B * self.corpus_size / self.avgdl)))
        return score

    def get_scores(self, document, average_idf):
        scores = []
        for index in xrange(self.corpus_size):
            score = self.get_score(document, index, average_idf)
            scores.append(score)
        return scores

###  가중치를 구해준다 .     
def get_bm25_weights(corpus):
    bm25 = BM25(corpus)
    average_idf = sum(map(lambda k: float(bm25.idf[k]), bm25.idf.keys())) / len(bm25.idf.keys())

    weights = []
    for doc in corpus:
        scores = bm25.get_scores(doc, average_idf)
        weights.append(scores)
    return weights    

In [43]:
data = pd.read_csv("data/태그별점추가.csv", encoding="utf-8")
data = data[["title","genre","synopsis","tag"]]
data

,title,genre,synopsis,tag
0,참교육,액션,무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다!<부활남> 채...,"블루스트링, 사이다, 학원물, 다크히어로, 먼치킨, 액션"
1,뷰티풀 군바리,드라마,'여자도 군대에 간다면?'본격 여자도 군대 가는 만화!,"직업드라마, 밀리터리, 드라마"
2,퀘스트지상주의,드라마,"[외모지상주의], [싸움독학], [인생존망]과 세계관을 공유하는 작품!공부, 싸움,...","게임판타지, 학원물, 드라마"
3,장씨세가 호위무사,무협/사극,‘당신이 부른 것이오. 나란 사람을... ’은둔고수 광휘. 호위무사 되다.웹소설 원...,"사이다, 힘숨찐, 판무, 먼치킨, 무협/사극, 소설원작"
4,윈드브레이커,스포츠,혼자서 자전거를 즐겨타던 모범생 조자현.원치 않게 자전거 크루의 일에 자꾸 휘말리게...,"스포츠성장, 스포츠"
...,...,...,...,...
556,밀실 마피아 게임,스릴러,"평범한 고등학생이었던 강철중, 어느 날 그가 잠에서 깨어나보니 처음 보는 밀실에 갇...","스릴러, 소설원작"
557,"아마도, 굿모닝",로맨스,"""시중은 내가 들죠. 당신이 기뻐할 때까지 최선을 다할 겁니다.""사업 확장을 위한 ...","로맨스, 소설원작"
558,천년간 노려왔습니다,로맨스,"용왕에게 토끼의 간을 바치지 못한 죄로, 계속해서 전생의 기억을 갖고 환생을 반복하...",로맨스
559,그림자 잡기,스릴러,"""모든 사건의 시작은 누군가 남겨 놓은 작은 흔적에 걸려 넘어지면서 시작된다. 하지...",스릴러


In [6]:
wt_title = data['title']
wt_title

0             참교육
1         뷰티풀 군바리
2         퀘스트지상주의
3       장씨세가 호위무사
4          윈드브레이커
          ...    
556     밀실 마피아 게임
557      아마도, 굿모닝
558    천년간 노려왔습니다
559        그림자 잡기
560          어떤소란
Name: title, Length: 561, dtype: object

In [7]:
data['sentence'] = data.apply(lambda row: ' '.join(row.values.astype(str)),axis=1)
data['sentence']

0      참교육 액션 무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다!...
1      뷰티풀 군바리 드라마 '여자도 군대에 간다면?'본격 여자도 군대 가는 만화! 직업드...
2      퀘스트지상주의 드라마 [외모지상주의], [싸움독학], [인생존망]과 세계관을 공유하...
3      장씨세가 호위무사 무협/사극 ‘당신이 부른 것이오. 나란 사람을... ’은둔고수 광...
4      윈드브레이커 스포츠 혼자서 자전거를 즐겨타던 모범생 조자현.원치 않게 자전거 크루의...
                             ...                        
556    밀실 마피아 게임 스릴러 평범한 고등학생이었던 강철중, 어느 날 그가 잠에서 깨어나...
557    아마도, 굿모닝 로맨스 "시중은 내가 들죠. 당신이 기뻐할 때까지 최선을 다할 겁니...
558    천년간 노려왔습니다 로맨스 용왕에게 토끼의 간을 바치지 못한 죄로, 계속해서 전생의...
559    그림자 잡기 스릴러 "모든 사건의 시작은 누군가 남겨 놓은 작은 흔적에 걸려 넘어지...
560    어떤소란 로맨스 스물셋 대학생 소란은, 크리스마스에 알바 대타를 나갔다가 꿈에 그리...
Name: sentence, Length: 561, dtype: object

In [8]:
def preprocessing(text):
    # 개행문자 제거
    text = re.sub('\\\\n', ' ', text)
    # # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

In [9]:
%time sentences=data['sentence'].apply(preprocessing)

Wall time: 8.98 ms


In [10]:
sentences = sentences +'.'

In [11]:
sentences

0      참교육 액션 무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다 ...
1      뷰티풀 군바리 드라마  여자도 군대에 간다면  본격 여자도 군대 가는 만화  직업드...
2      퀘스트지상주의 드라마  외모지상주의    싸움독학    인생존망 과 세계관을 공유하...
3      장씨세가 호위무사 무협 사극  당신이 부른 것이오  나란 사람을     은둔고수 광...
4      윈드브레이커 스포츠 혼자서 자전거를 즐겨타던 모범생 조자현 원치 않게 자전거 크루의...
                             ...                        
556    밀실 마피아 게임 스릴러 평범한 고등학생이었던 강철중  어느 날 그가 잠에서 깨어나...
557    아마도  굿모닝 로맨스  시중은 내가 들죠  당신이 기뻐할 때까지 최선을 다할 겁니...
558    천년간 노려왔습니다 로맨스 용왕에게 토끼의 간을 바치지 못한 죄로  계속해서 전생의...
559    그림자 잡기 스릴러  모든 사건의 시작은 누군가 남겨 놓은 작은 흔적에 걸려 넘어지...
560    어떤소란 로맨스 스물셋 대학생 소란은  크리스마스에 알바 대타를 나갔다가 꿈에 그리...
Name: sentence, Length: 561, dtype: object

In [12]:
import gensim

model = gensim.models.Word2Vec.load('ko_webtoon.bin')

In [13]:
model.wv.most_similar("후계")

[('황태자', 0.46964365243911743),
 ('계승', 0.4495263695716858),
 ('숙부', 0.44683563709259033),
 ('장남', 0.439852774143219),
 ('서자', 0.43450576066970825),
 ('정실', 0.43223437666893005),
 ('왕좌', 0.426585853099823),
 ('영지', 0.41764959692955017),
 ('삼촌', 0.4162304997444153),
 ('왕세자', 0.4153168797492981)]

In [14]:
webtoonlist = []
okt = Okt()

for i in range(len(sentences)):
    nouns = okt.nouns(sentences[i])
    webtoonlist.append(nouns)

webtoonlist

[['교육',
  '액션',
  '교권',
  '위해',
  '교권',
  '보호국',
  '소속',
  '나화',
  '진의',
  '교육',
  '시작',
  '부활',
  '남',
  '채용',
  '택',
  '작가',
  '기녀',
  '한가람',
  '작가',
  '신작',
  '블루스',
  '트링',
  '사이다',
  '학원물',
  '다크',
  '히어로',
  '먼치킨',
  '액션'],
 ['뷰티풀',
  '군',
  '바리',
  '드라마',
  '여자도',
  '군대',
  '간다',
  '본격',
  '여자도',
  '군대',
  '만화',
  '직업',
  '드라마',
  '밀리터리',
  '드라마'],
 ['퀘스트',
  '지상',
  '주의',
  '드라마',
  '외모',
  '지상',
  '주의',
  '싸움',
  '독학',
  '인생',
  '존망',
  '과',
  '세계관',
  '공유',
  '작품',
  '공부',
  '싸움',
  '외모',
  '뭐',
  '하나',
  '것',
  '고등학생',
  '김수현',
  '의',
  '눈앞',
  '갑자기',
  '퀘스트',
  '창',
  '첫',
  '퀘스트',
  '엄마',
  '사과',
  '를',
  '완료',
  '수현',
  '보상',
  '키',
  '퀘스트',
  '퀘스트',
  '완료',
  '다음',
  '퀘스트',
  '학교',
  '일진',
  '녀',
  '키스',
  '게임판타지',
  '학원물',
  '드라마'],
 ['장씨',
  '세',
  '호위',
  '무사',
  '무협',
  '사극',
  '당신',
  '것',
  '이오',
  '사람',
  '은둔',
  '고수',
  '광휘',
  '호위',
  '무사',
  '웹소설',
  '원작',
  '웰',
  '메이드',
  '무협',
  '시대극',
  '사이다',
  '힘숨',
  '판무',
  '먼치킨',
  '무협',
  '사극',
  '소설',
  '원작'],
 ['윈드브

In [15]:
max_score = []
mean_score = []
k = 0
l = 4
for i in range(len(webtoonlist[k])):
    for j in range(len(webtoonlist[l])):
        try:
            score = model.wv.similarity(webtoonlist[k][i], webtoonlist[l][j])
            if(len(max_score) == 10):
                if(min(max_score) < score):
                    max_score.remove(min(max_score))
                    max_score.append(score)
            else:
                max_score.append(score)
        except:
            continue
mean_score.append(sum(max_score)/len(max_score))

In [49]:
mean_score

[0.5759055852890015]

In [57]:
for i in range(len(webtoonlist[0])):
    try:
        print(model.wv.similarity(webtoonlist[0][i], webtoonlist[2][j]))
    except:
        continue
    

-0.03832065
0.054907396
0.3759329
-0.07780954
0.07867822
-0.048433132
0.20284826
0.09853201
-0.14348829
-0.15054148
0.21840237
-0.03375193
0.13950546
0.16411726
-0.026928104
0.086352706
-0.068455644
0.08479902
-0.13675703
0.102228306
0.10419598
0.07101808
0.023242574
0.016020708
0.45548624
-0.07521133
0.033203125
0.07679662
0.11770235
-0.03832065
0.009328432
0.22256793
-0.016784899
0.0566902
0.009199418
0.11411476
-0.052561224
0.23104975
0.086723484
0.2761069
0.10068062
-0.01601278
-0.07174256
0.2102702
0.16716668
0.19998603
-0.07347177


In [63]:
score_list={'webtoon1':[],'webtoon2':[], 'similarity':[]}

for k in range(len(webtoonlist)):
    for i in range(k + 1, len(webtoonlist)):
        for j in range(len(webtoonlist[k])):
            max_score = []
            for l in range(len(webtoonlist[i])):
                try:
                    score = model.wv.similarity(webtoonlist[k][j], webtoonlist[i][l])
                    if(len(max_score) == 10):
                        if(min(max_score) < score):
                            max_score.remove(min(max_score))
                            max_score.append(score)
                    else:
                        max_score.append(score)
                except:
                    continue
        score_list['webtoon1'].append(wt_title[k])
        score_list['webtoon2'].append(wt_title[i])
        score_list['similarity'].append(sum(max_score)/len(max_score))
        

In [36]:
max_score

[0.6342883,
 0.12184936,
 0.3792057,
 0.26917166,
 0.34031373,
 0.18497744,
 0.55000246,
 0.6342883,
 0.6342883,
 0.6342883]

In [64]:
df = pd.DataFrame(score_list)
df

,webtoon1,webtoon2,similarity
0,참교육,뷰티풀 군바리,0.217214
1,참교육,퀘스트지상주의,0.434063
2,참교육,장씨세가 호위무사,0.406448
3,참교육,윈드브레이커,0.230949
4,참교육,팔이피플,0.311559
...,...,...,...
157075,"아마도, 굿모닝",그림자 잡기,0.265969
157076,"아마도, 굿모닝",어떤소란,0.303380
157077,천년간 노려왔습니다,그림자 잡기,0.321381
157078,천년간 노려왔습니다,어떤소란,0.619994


In [65]:
df.to_csv("data/웹툰 시놉시스 관련 유사도.csv")

In [57]:
simil = pd.read_csv("data/웹툰 시놉시스 유사도.csv", encoding="cp949")
simil

C:\Users\smhrd\anaconda3\envs\opencv3\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,webtoon1,webtoon2,titles,similarity
0,0,참교육,NaN,"0,1",0.217214
1,1,참교육,NaN,"0,2",0.434063
2,2,참교육,NaN,"0,3",0.406448
3,3,참교육,NaN,"0,4",0.230949
4,4,참교육,NaN,"0,5",0.311559
...,...,...,...,...,...
157075,157075,NaN,NaN,"557,559",0.265969
157076,157076,NaN,NaN,"557,560",0.303380
157077,157077,NaN,NaN,"558,559",0.321381
157078,157078,NaN,NaN,"558,560",0.619994


In [59]:
for i in range (len(simil)):
    if "9," in simil["titles"]:
        simil["webtoon1"]=data["title"][9]

In [20]:
sum(max_score)/len(max_score)

0.8704367756843567

In [167]:
# 코사인 유사도 상위 10개 뽑아내기
wtlist = []
wtlist_score = []
for i in range(len(webtoonlist)):
    for j in range(len(webtoonlist[i])):
        try:
            word_top = model.wv.most_similar(positive=webtoonlist[i][j], topn=10)
            wtlist.append([w[0] for w in word_top]) # 코사인 유사도 단어 추출
            wtlist_score.append([w[1] for w in word_top])
        except:
            wtlist.append([])
            wtlist_score.append([0])
            continue

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [168]:
wtlist

[['수업', '훈련', '과외', '양성', '연구', '상담', '학교', '사회', '의학', '실천'],
 ['어드벤처', '게임', '퍼즐', '격투', '미소녀', '판타지', '파이터', '외전', '리얼', '어택'],
 ['폭력', '사회', '도덕', '종교', '서방', '낭만', '사이비', '정치', '유교', '여성'],
 ['통해', '예방', '방해', '감시', '재난', '거부', '해킹', '보호', '목적', '환경'],
 ['폭력', '사회', '도덕', '종교', '서방', '낭만', '사이비', '정치', '유교', '여성'],
 [],
 ['출신', '선임', '결성', '설립', '스카우트', '회장', '파견', '수장', '소장', '구성'],
 [],
 [],
 ['수업', '훈련', '과외', '양성', '연구', '상담', '학교', '사회', '의학', '실천'],
 ['종료', '마무리', '진행', '완료', '전래', '이전', '완성', '전개', '처음', '설립'],
 ['탄생', '복귀', '타락', '회복', '각성', '좌절', '출현', '완성', '부흥', '승천'],
 ['무주', '담', '살', '북', '스트릿', '은진', '지은', '삼류', '석호', '김재'],
 ['사용', '활용', '설계', '대체', '투입', '전환', '이용', '제조', '공개', '모집'],
 ['혁', '욱', '신영', '이재', '웅', '우영', '김', '진우', '희영', '정호'],
 ['소설가', '만화가', '사진작가', '소설', '여배우', '거장', '디자이너', '피아니스트', '아티스트', '작품'],
 ['선비', '선녀', '여인', '첩', '부자', '제자', '무당', '후배', '선배', '건달'],
 [],
 ['소설가', '만화가', '사진작가', '소설', '여배우', '거장', '디자이너', '피아니스트', '아티스트', '작품'],
 ['단편', 

In [169]:
wtlist_score

[[0.5871355533599854,
  0.5792600512504578,
  0.522475004196167,
  0.5201014876365662,
  0.5097616910934448,
  0.5069489479064941,
  0.49593883752822876,
  0.4939671754837036,
  0.4921710789203644,
  0.49198848009109497],
 [0.7256604433059692,
  0.6752271056175232,
  0.6438050270080566,
  0.6437793970108032,
  0.6113677024841309,
  0.6075881719589233,
  0.5925247669219971,
  0.5913335084915161,
  0.5901573896408081,
  0.552312970161438],
 [0.5033485889434814,
  0.4867202639579773,
  0.4633599519729614,
  0.43974536657333374,
  0.4268415570259094,
  0.42390549182891846,
  0.4177086651325226,
  0.41480565071105957,
  0.40768903493881226,
  0.40392208099365234],
 [0.47746923565864563,
  0.396087646484375,
  0.3603771924972534,
  0.36002999544143677,
  0.3438657522201538,
  0.33485209941864014,
  0.3332948684692383,
  0.33040887117385864,
  0.325563907623291,
  0.32376378774642944],
 [0.5033485889434814,
  0.4867202639579773,
  0.4633599519729614,
  0.43974536657333374,
  0.426841557025909

In [17]:
word_top

[('코미디', 0.721144437789917),
 ('로맨틱', 0.7021660804748535),
 ('스릴러', 0.6989323496818542),
 ('영화', 0.6428799629211426),
 ('드라마', 0.6342883110046387),
 ('시트콤', 0.619952917098999),
 ('미스터리', 0.6108185648918152),
 ('소설', 0.6032933592796326),
 ('연애', 0.6029824614524841),
 ('사극', 0.590453028678894)]

In [18]:
def get_model(test_wordList, model_name,tg_num):
    ## 모델 로드하기 
    model = gensim.models.Word2Vec.load(model_name)

    ## most_similar 는 코사인 유사도를 구해준다. 
#     word_top = embedding_model.wv.most_similar(positive= test_wordList, topn=5)  # 코사인 유사도 상위 5개를 뽑아 내기 , 단어기준 
    word_top = model.most_similar(test_wordList , topn=tg_num) # 코사인 유사도 상위 5개를 뽑아 내기 , 단어기준 
    wtli = []
    wtli = [w[0] for w in word_top]                    # 코사인 유사도 단어 추출 
    wtli = sorted(wtli, reverse=True)
    wtli_score = [w[1] for w in word_top]              # 코사인 유사도 스코어 추출 
    wtli_score = sorted(wtli_score, reverse=True)
    wtli_score = [round(x,2) for x in wtli_score]
    return wtli ,wtli_score

In [139]:
wtli  = []
wtli_score = []

## 모델명 지정하기 
model_name = 'ko_webtoon.bin'
tg_num = 10
test_wordList = ['교육',
  '액션',
  '교권',
  '위해',
  '교권',
  '소속',
  '교육',
  '시작',
  '부활',
  '남',
  '채용',
  '택',
  '작가',
  '기녀',
  '작가',
  '신작',
  '블루스',
  '트링',
  '사이다',
  '학원물',
  '다크',
  '히어로',
  '먼치킨',
  '액션']     # 유사도 측정 담어 입력 

# train_model(test_wordList,model_name)
wtli,wtli_score= get_model(test_wordList,model_name,tg_num)  ## 테스트리스트를 , 모델에서 , 몇개 뽑기 


print(wtli)
print(wtli_score)

['힘숨', '판타지', '코믹스', '코믹', '코미디', '외전', '어드벤처', '만화', '러브', '개그']
[0.61, 0.6, 0.55, 0.55, 0.55, 0.55, 0.54, 0.53, 0.53, 0.53]


C:\Users\smhrd\anaconda3\envs\opencv3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


In [140]:
class Cleaning_Text:
    def token_word(text):
        text = word_tokenize(text)
        if len(text) > 1:
            text = text
        else:
            pass
        text_token = text
        return text_token
    
    def token_sent(text):
        text = sent_tokenize(text)
        if len(text) > 1:
            text = text
        else:
            pass
        text_token = text
        return text_token        
    
    def listToText(inputList):
        returnText = ''
        for i in inputList:
            returnText = returnText + i
        rt2 = Cleaning_Text.text_cleaning(returnText)
        return rt2

    ##  # step_4,   공통 코드 , 텍스트 클리닝 
    def text_cleaning(text): 
    ##################################  gensim 사용을 위한 정규표현식 200624 
        hangul_path9 = '[가-힣]+\.'                  # 한글로 포함되다 . 

        hangul_path0 = '[가-힣]+\.[가-힣]{1}'        # 한글 . + 한글 처리 
        hangul_path1 = '[가-힣]+\.[\d]{1}'           # 한글 . + 숫자 처리 [0-9]
        hangul_path2 = '[가-힣]+\.[a-z]{1}'          # 한글 . + 영어 소문자 
        hangul_path3 = '[가-힣]+\.[A-Z]{1}'          # 한글 . + 영어 대문자 
        hangul_path4 = '[가-힣]+\.[\S]{1}'           # 한글 . + 비공백 [^ \t\n\r\f\v]와 같다.    
        hangul_path5 = '[가-힣]+\.[\s]{1}'           # 한글 . + 공백 [ \t\n\r\f\v]와 같다.
        hangul_path6 = '[가-힣]+\.[\W]{1}'           # 한글 . + 숫자 + 문자가 아닌 것 [^a-zA-Z0-9]와 같다.
        hangul_path7 = '[가-힣]+\.[\w]{1}'           # 한글 . + 숫자 + 문자 [a-zA-Z0-9]와 같다.
        hangul_path8 = '[가-힣]+\.[\b]{1}'           # 한글 . + 단어 경계 (`\w`와 `\W`의 경계)

        reg_path = hangul_path0 + '|' + hangul_path1 + '|'+ hangul_path2 + '|'+ hangul_path3 + '|'+ hangul_path4+ '|'+ hangul_path5
        hangul = re.compile(reg_path)              # 한글 + 모모로 포함되다 . 

        result = hangul.findall(text)                                                   # 정규식에 일치되는 부분을 리스트 형태로 저장, 단어 반환 
        result = list(set(result))    
        for x in result:
            text = text.replace(x, x[:-1] + '\n' + x[-1:])

        ##################################  gensim 사용을 위한 정규표현식 200624
        text = re.sub('\[.+?\]','', text)         # 대괄호 [] 이내 모든 문자 삭제     

        return text 

In [154]:
query  = []
query_score = []

## 모델명 지정하기 
model_name = 'ko_webtoon.bin'
tg_num = 10
test_wordList = [
  '살수',
  '판타지',
  '살',
  '노소',
  '그',
  '정체',
  '전설',
  '후계',
  '이제',
  '비서',
  '경',
  '향',
  '살',
  '수행',
  '시작',
  '판무',
  '판타지']

## 모델 로드하고 값 추출하기 
query,query_score= get_model(test_wordList,model_name,tg_num)      ## 테스트리스트(유사단어)를 , 모델에서 , 몇개 뽑기 

ct_tolist = []
ct_tolist = sentences.tolist()
ct_text = Cleaning_Text.listToText(ct_tolist)
textList  = Cleaning_Text.token_sent(ct_text)
tok_corpus = [Cleaning_Text.token_word(x) for x in textList]
#############################
bm25Model = BM25(tok_corpus)
bm25      = BM25Okapi(tok_corpus)

#############################
average_idf = sum(map(lambda k: float(bm25Model.idf[k]), bm25Model.idf.keys())) / len(bm25Model.idf.keys())
scores = bm25Model.get_scores(query,average_idf)
idx = scores.index(max(scores))                                           # 0 번재에 가장 높은 값이 있음 
tList = textList[idx]


# print(idx, tList)         # 가장 높은 유사도 인덱스, 텍스트 , 1개 뽑아내기 
print(query,query_score)

['환생', '현세', '죽', '주인공', '전생', '이야기', '어머니', '아내', '소설', '먼치킨'] [0.46, 0.46, 0.45, 0.45, 0.44, 0.44, 0.44, 0.43, 0.43, 0.43]


C:\Users\smhrd\anaconda3\envs\opencv3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


In [142]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\smhrd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [161]:
# 1. 스코어  뽑기 
print(query_score)

# 2. n개 코사인 유사도 문장 뽑기
cnt = len(query)
bm25_okapi = bm25.get_top_n(query, textList, n=10)    # 요약문장 
print(bm25_okapi)

# 3. 돌아가는 시간 체크하기 
ends = int(time.time()) - start_now
ends = '{} 초'.format(ends)

# 4. 테스트 결과 기록하기

dic = {'웹툰제목' : wt_title[7],
       '입력데이터':test_wordList,
       '출력데이터':query,
       '점수':query_score,
       '유사도문장':bm25_okapi,
       '소요시간':ends
      }

print('성공')
print(ends)

[0.46, 0.46, 0.45, 0.45, 0.44, 0.44, 0.44, 0.43, 0.43, 0.43]
['전지적 독자 시점 판타지  이건 내가 아는 그 전개다 한순간에 세계가 멸망하고  새로운 세상이 펼쳐졌다 오직 나만이 완주했던 소설 세계에서 평범했던 독자의 새로운 삶이 시작된다  서바이벌  세계관  레드아이스 스튜디오  아포칼립스  게임판타지  헌터물  이능력배틀물  소년왕도물  빙의  먼치킨  두뇌싸움  구원서사  판타지  소설원작.', '용두사망 소설 속의 악녀가 되었다 로맨스 엔딩 직전 느닷없이 제국이 멸망하는  용두사망  소설 속 악녀에게 빙의했다  마계 침략으로부터 살아남기 위해 망명을 계획하지만침략의 원흉이자 마왕의 정체를 숨기고 있는 남자  아르비스 대공이   년을 기다렸다며 애절한 고백을 털어 놓는데     나  이 용두사망 소설 속에서 살아남을 수 있을까  빙의  로판  로맨스  소설원작.', '행운을 빌어요  용사님  로맨스 전생에 무찌른 마왕이 현생의 직장 상사  마왕을 무찌른 용사의 기억을 가지고 다시 태어난 평범한 직장인 백영웅 용사의 기억이 도움이 된 건 어릴 적 웹소설을 썼을 때뿐  영웅은 현실에서 쓸모없는 전생의 기억을 아쉬워하며 가까스로 대기업인 대룡 ENT의 대표 이사 비서실에 취직한다  그런데 처음 만난 대표 이사 도연화에게 전생에서 경험한 적 있는 긴장감과 떨림을 느끼는데   이건 마치  아주 강한 마수를 마주했을 때와 같아   전생 마왕  대표 이사와 전생 용사  말단 비서의 지독하게 얽힌 로맨스  회귀  사내연애  로맨스.', '최면학교 개그  지금부터 당신은 최면에 빠집니다    최면능력을 가진 주인공 김찬규는 여자에게 인기를 얻고 싶어 열심히 최면을 이용해보지만 매번 뜻대로 되지 않는다  거듭되는 실패 속에 만난 비범한 능력의 소녀와 김찬규  특별하지만 안타까운 그들의 이야기  이것은 개그만화가 아닙니다  병맛  개그.', '낙향문사전 무협 사극 온갖 비리가 가득한 과거 시험에 낙방해 좌절한 주인공 손빈  

In [156]:
bm25_okapi[0]

'전지적 독자 시점 판타지  이건 내가 아는 그 전개다 한순간에 세계가 멸망하고  새로운 세상이 펼쳐졌다 오직 나만이 완주했던 소설 세계에서 평범했던 독자의 새로운 삶이 시작된다  서바이벌  세계관  레드아이스 스튜디오  아포칼립스  게임판타지  헌터물  이능력배틀물  소년왕도물  빙의  먼치킨  두뇌싸움  구원서사  판타지  소설원작.'

In [162]:
dic

{'웹툰제목': '앵무살수',
 '입력데이터': ['살수',
  '판타지',
  '살',
  '노소',
  '그',
  '정체',
  '전설',
  '후계',
  '이제',
  '비서',
  '경',
  '향',
  '살',
  '수행',
  '시작',
  '판무',
  '판타지'],
 '출력데이터': ['환생', '현세', '죽', '주인공', '전생', '이야기', '어머니', '아내', '소설', '먼치킨'],
 '점수': [0.46, 0.46, 0.45, 0.45, 0.44, 0.44, 0.44, 0.43, 0.43, 0.43],
 '유사도문장': ['전지적 독자 시점 판타지  이건 내가 아는 그 전개다 한순간에 세계가 멸망하고  새로운 세상이 펼쳐졌다 오직 나만이 완주했던 소설 세계에서 평범했던 독자의 새로운 삶이 시작된다  서바이벌  세계관  레드아이스 스튜디오  아포칼립스  게임판타지  헌터물  이능력배틀물  소년왕도물  빙의  먼치킨  두뇌싸움  구원서사  판타지  소설원작.',
  '용두사망 소설 속의 악녀가 되었다 로맨스 엔딩 직전 느닷없이 제국이 멸망하는  용두사망  소설 속 악녀에게 빙의했다  마계 침략으로부터 살아남기 위해 망명을 계획하지만침략의 원흉이자 마왕의 정체를 숨기고 있는 남자  아르비스 대공이   년을 기다렸다며 애절한 고백을 털어 놓는데     나  이 용두사망 소설 속에서 살아남을 수 있을까  빙의  로판  로맨스  소설원작.',
  '행운을 빌어요  용사님  로맨스 전생에 무찌른 마왕이 현생의 직장 상사  마왕을 무찌른 용사의 기억을 가지고 다시 태어난 평범한 직장인 백영웅 용사의 기억이 도움이 된 건 어릴 적 웹소설을 썼을 때뿐  영웅은 현실에서 쓸모없는 전생의 기억을 아쉬워하며 가까스로 대기업인 대룡 ENT의 대표 이사 비서실에 취직한다  그런데 처음 만난 대표 이사 도연화에게 전생에서 경험한 적 있는 긴장감과 떨림을 느끼는데   이건 마치  아주 강한 마수를 마주했을 때와 같아   전생 마왕  대표 이사와 전생 용사

In [148]:
scores

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1.5901305145710047,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1.1612901651181444,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1.5901305145710047,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.595771368880906,
 0,
 0.595771368880906,
 0,
 0,
 0,
 0,
 1.1612901651181444,
 0,
 0,
 0,
 1.1612901651181444,
 0.7720640561240426,
 0,
 0,
 0,
 0,
 0.595771368880906,
 0,
 0,
 0,
 0,
 0.9905342636016224,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.9048128588053593,
 0,
 0,
 0,
 0.8059214272865811,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.8482471560356635,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.595771368880906,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1.1612901651181444,
 0,
 0,
 0,
 0,
 1.1612901651181444,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.595771368880906,
 0.595771368880906,
 0,
